# Import

In [4]:

# BOILER PLATE, MUST BE RUN ON SUBMIT NODE
%load_ext autoreload
%autoreload 2
import socket
print("Running on host: ", socket.gethostname())

import sys 
lib_path = '/fsx_0/user/tranx/experiments'
if lib_path not in sys.path:
    sys.path.append(lib_path)
#=================================================
    
import plotly.express as px
import plotly.graph_objects as go
import re
import time
import shutil
import json
import glob
import pandas as pd
import os

from lib import eval_helper
from lib import utils
from lib import slurm
from lib.slurm import run_sbatch_job


Running on host:  submit-1


In [4]:
32*4*160*8*(10200 - 5800)/(8*1e6)

90.112

In [6]:
params = {
    "eval_plan": "Llama31_336px",
    "json_config": "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_vqa.json",
    "checkpoint_dir": "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch",
    "benchmark_name": "vqa",
    "checkpoint_id": "2000"
}

job = run_job(
    sbatch_base_script="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    sbatch_overwrite={
        "job-name": "eval_" + params["benchmark_name"]
    },
    positional_env_vars=list(params.values())
)

job

sbatch --parsable --job-name=eval_vqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_vqa.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch vqa 2000


9675

# Eval plan

## Llama31

In [42]:
# Llama31
EVAL_PLAN = "Llama31_336px"
BASE_SCRIPT = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
CONFIG_DIR_LLAMA31 = "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31"
CHECKPOINT_DIR_LLAMA31 = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch"

all_benchmarks = "mmmu docvqa mathvista ai2d chartqa vqa textvqa infographics_w_ocr infographics mmbench"
checkpoint_start = 2000
checkpoint_end = 8000
checkpoint_int = 1000

job_dict = {}
for benchmark in all_benchmarks.split():
    job_dict[benchmark] = {}

    for chk in range(checkpoint_start, checkpoint_end + checkpoint_int, checkpoint_int):
        params = {
            "eval_plan": EVAL_PLAN,
            "json_config": f"{CONFIG_DIR_LLAMA31}/eval_{benchmark}.json",
            "checkpoint_dir": CHECKPOINT_DIR_LLAMA31,
            "benchmark_name": benchmark,
            "checkpoint_id": str(chk)
        }

        assert os.path.exists(params["json_config"])
        assert os.path.exists(f"{params['checkpoint_dir']}/checkpoint-{chk}")

        job_id = run_sbatch_job(
            sbatch_base_script=BASE_SCRIPT,
            sbatch_overwrite={
                "job-name": f"eval_{benchmark}"
            },
            positional_env_vars=list(params.values())
        )

        job_dict[benchmark][chk] = int(job_id)

with open(f'job_dict_{EVAL_PLAN}.json', 'w') as f:
    json.dump(job_dict, f, indent=4)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 2000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 3000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 4000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json 

In [113]:
# Llama31
EVAL_PLAN = "Llama31_336px_8500_9500"
checkpoints = [8500, 9000, 9500]

job_dict_json = f'job_dict_{EVAL_PLAN}.json'
assert not os.path.exists(job_dict_json)

BASE_SCRIPT = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
CONFIG_DIR_LLAMA31 = "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31"
CHECKPOINT_DIR_LLAMA31 = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch"

all_benchmarks = "mmmu docvqa mathvista ai2d chartqa vqa textvqa infographics_w_ocr infographics mmbench"

job_dict = {}
for benchmark in all_benchmarks.split():
    job_dict[benchmark] = {}

    for chk in checkpoints:
        params = {
            "eval_plan": EVAL_PLAN,
            "json_config": f"{CONFIG_DIR_LLAMA31}/eval_{benchmark}.json",
            "checkpoint_dir": CHECKPOINT_DIR_LLAMA31,
            "benchmark_name": benchmark,
            "checkpoint_id": str(chk)
        }

        assert os.path.exists(params["json_config"])
        assert os.path.exists(f"{params['checkpoint_dir']}/checkpoint-{chk}")

        job_id = run_job(
            sbatch_base_script=BASE_SCRIPT,
            sbatch_overwrite={
                "job-name": f"eval_{benchmark}"
            },
            positional_env_vars=list(params.values())
        )

        job_dict[benchmark][chk] = int(job_id)

with open(job_dict_json, 'w') as f:
    json.dump(job_dict, f, indent=4)

print(f"Saved job_dict to {job_dict_json}")

AssertionError: 

In [73]:
job_dict

{'mmmu': {2000: 9759,
  3000: 9760,
  4000: 9761,
  5000: 9762,
  6000: 9763,
  7000: 9764,
  8000: 9765},
 'docvqa': {2000: 9766,
  3000: 9767,
  4000: 9768,
  5000: 9769,
  6000: 9770,
  7000: 9771,
  8000: 9772},
 'mathvista': {2000: 9773,
  3000: 9774,
  4000: 9775,
  5000: 9776,
  6000: 9777,
  7000: 9778,
  8000: 9779},
 'ai2d': {2000: 9780,
  3000: 9781,
  4000: 9782,
  5000: 9783,
  6000: 9784,
  7000: 9785,
  8000: 9786},
 'chartqa': {2000: 9787,
  3000: 9788,
  4000: 9789,
  5000: 9790,
  6000: 9791,
  7000: 9792,
  8000: 9793},
 'vqa': {2000: 9794,
  3000: 9795,
  4000: 9796,
  5000: 9797,
  6000: 9798,
  7000: 9799,
  8000: 9800},
 'textvqa': {2000: 9801,
  3000: 9802,
  4000: 9803,
  5000: 9804,
  6000: 9805,
  7000: 9806,
  8000: 9807},
 'infographics_w_ocr': {2000: 9808,
  3000: 9809,
  4000: 9810,
  5000: 9811,
  6000: 9812,
  7000: 9813,
  8000: 9814},
 'infographics': {2000: 9815,
  3000: 9816,
  4000: 9817,
  5000: 9818,
  6000: 9819,
  7000: 9820,
  8000: 9821},
 'm

## Llama31 - 10000

In [3]:
job_dict_lm31_10k = eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    eval_plan="Llama31_10k",
    eval_config_dir="/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31",
    checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch",
    checkpoints=[10000],
    # benchmarks=["mathvista"]
)

job_dict_lm31_10k

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 10000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch docvqa 10000
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mathvista 10000
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31

{'mmmu': {10000: 10008},
 'docvqa': {10000: 10009},
 'mathvista': {10000: 10010},
 'ai2d': {10000: 10011},
 'chartqa': {10000: 10012},
 'vqa': {10000: 10013},
 'textvqa': {10000: 10014},
 'infographics_w_ocr': {10000: 10015},
 'infographics': {10000: 10016},
 'mmbench': {10000: 10017}}

## Llama31 - fb

In [6]:
job_dict_fb_lm31 = eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    eval_plan="FBL_Llama31",
    eval_config_dir="/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31",
    checkpoint_dir="/fsx_0/checkpoints/tranx/fb_llama3.1",
    checkpoints=[8000, 10000],
    # benchmarks=["mathvista"]
)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/fb_llama3.1 mmmu 8000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/fb_llama3.1 mmmu 10000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_docvqa.json /fsx_0/checkpoints/tranx/fb_llama3.1 docvqa 8000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_docvqa.json /fsx_0/checkpoints/tranx/fb_llama3.1 docvqa 10000
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experime

## MH19

In [1]:
checkpoints = [10500 + i*1000 for i in range(9)]
checkpoints

[10500, 11500, 12500, 13500, 14500, 15500, 16500, 17500, 18500]

In [3]:
job_dict_mh19 = eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    eval_plan="MH19_10500_18500",
    eval_config_dir="/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval",
    checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/MH19_336px_128nodes_bz32_resume",
    checkpoints=[10500 + i*1000 for i in range(9)],
    # benchmarks=["mathvista"]
)

RuntimeError: Found existing job_dict: eval/logs/job_dict_MH19_10500_18500.json

In [12]:
df = eval_helper.get_eval_scores(job_dict_mh19)

Got result for mmmu - 10500
Got result for mmmu - 11500
Got result for mmmu - 12500
Got result for mmmu - 13500
Got result for mmmu - 14500
Got result for mmmu - 15500
Got result for mmmu - 16500
Got result for mmmu - 17500
Got result for mmmu - 18500
Got result for docvqa - 10500
Got result for docvqa - 11500
Got result for docvqa - 12500
Got result for docvqa - 13500
Got result for docvqa - 14500
Got result for docvqa - 15500
Got result for docvqa - 16500
Got result for docvqa - 17500
Got result for docvqa - 18500
Got result for mathvista - 10500
Got result for mathvista - 11500
Got result for mathvista - 12500
Got result for mathvista - 13500
Got result for mathvista - 14500
Got result for mathvista - 15500
Got result for mathvista - 16500
Got result for mathvista - 17500
Got result for mathvista - 18500
Got result for ai2d - 10500
Got result for ai2d - 11500
Got result for ai2d - 12500
Got result for ai2d - 13500
Got result for ai2d - 14500
Got result for ai2d - 15500
Got result fo

In [13]:
df.loc[17500]

mmmu/accuracy                                    0.5878
mmmu/mllm_eval_accuracy                          0.5889
docvqa/anls_total_score                          0.6201
docvqa/mllm_evaluation_anls_score                0.6216
mathvista/accuracy                                0.377
ai2d/accuracy                                    0.7704
chartqa/accuracy                                 0.4914
vqa/accuracy                                     0.7035
vqa/mllm_evaluation_accuracy                     0.7256
textvqa/accuracy                                 66.898
textvqa/mllm_eval_accuracy                       71.578
infographics_w_ocr/anls_total_score              0.6256
infographics_w_ocr/mllm_evaluation_anls_score    0.5712
infographics/anls_total_score                    0.4798
infographics/mllm_evaluation_anls_score          0.4177
mmbench/overall                                  0.7422
Name: 17500, dtype: object

# Parse results

In [1]:
# job_dict_file = 'job_dict_Llama31_336px.json'
job_dict_file = 'job_dict_Llama31_336px_8500_9500.json'

with open(job_dict_file, 'r') as f:
    job_dict = json.load(f)

df = eval_helper.get_eval_scores(job_dict)
df

NameError: name 'json' is not defined

In [4]:
job_dict_file = 'job_dict_FBL_Llama31.json'
with open(job_dict_file, 'r') as f:
    job_dict = json.load(f)

df2 = eval_helper.get_eval_scores(job_dict)
df2

Got result for mmmu - 8000
Got result for mmmu - 10000
Got result for docvqa - 8000
Got result for docvqa - 10000
Got result for mathvista - 8000
Got result for mathvista - 10000
Got result for ai2d - 8000
Got result for ai2d - 10000
Got result for chartqa - 8000
Got result for chartqa - 10000
Got result for vqa - 8000
Got result for vqa - 10000
Got result for textvqa - 8000
Got result for textvqa - 10000
Got result for infographics_w_ocr - 8000
Got result for infographics_w_ocr - 10000
Got result for infographics - 8000
Got result for infographics - 10000
Got result for mmbench - 8000
Got result for mmbench - 10000


,mmmu/accuracy,mmmu/mllm_eval_accuracy,docvqa/anls_total_score,docvqa/mllm_evaluation_anls_score,mathvista/accuracy,ai2d/accuracy,chartqa/accuracy,vqa/accuracy,vqa/mllm_evaluation_accuracy,textvqa/accuracy,textvqa/mllm_eval_accuracy,infographics_w_ocr/anls_total_score,infographics_w_ocr/mllm_evaluation_anls_score,infographics/anls_total_score,infographics/mllm_evaluation_anls_score,mmbench/overall
8000,0.5067,0.5156,0.5386,0.5411,0.344,0.7121,0.4746,0.6831,0.7137,59.028,65.38,0.5902,0.536,0.4384,0.3746,0.6635
10000,0.4744,0.4822,0.54,0.5432,0.348,0.6953,0.4578,0.6931,0.7224,59.414,65.708,0.592,0.5354,0.4472,0.3785,0.6772


# Stage 2

In [2]:
EVAL_SBATCH = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
# ALIGNER_CODE_DIR="/fsx_0/user/tranx/eval_adel"
# EVAL_CONFIG_DIR=f"/fsx_0/user/tranx/eval_adel/llm_mm_aligner/experiments/aws_adel/eval"

ALIGNER_CODE_DIR="/fsx_0/user/tranx/adel_prod"
EVAL_CONFIG_DIR=f"/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval"

# EVAL_CONFIG_DIR=f"/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval"

# STAGE2_OUTPUT_DIR="/fsx_0/checkpoints/tranx/MM9-Stage2-70B"

## Exp 29

In [73]:
eval_helper.run_eval_sweep(
    output_dir=f"/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp",
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR,
    aligner_parent_dir=ALIGNER_CODE_DIR,
)

New checkpoints: []


In [21]:
eval_helper.get_eval_scores_all("/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp")

[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000, 4200, 4400, 4600, 4800]
Got result for mmmu - 200: [('mmmu/accounting', 0.4), ('mmmu/agriculture', 0.6333333333333333), ('mmmu/architecture_and_engineering', 0.23333333333333334), ('mmmu/art', 0.6666666666666666), ('mmmu/art_theory', 0.9333333333333333), ('mmmu/basic_medical_science', 0.7), ('mmmu/biology', 0.43333333333333335), ('mmmu/chemistry', 0.4), ('mmmu/clinical_medicine', 0.6), ('mmmu/computer_science', 0.6), ('mmmu/design', 0.7666666666666667), ('mmmu/diagnostics_and_laboratory_medicine', 0.36666666666666664), ('mmmu/economics', 0.6666666666666666), ('mmmu/electronics', 0.26666666666666666), ('mmmu/energy_and_power', 0.5333333333333333), ('mmmu/finance', 0.5333333333333333), ('mmmu/geography', 0.5333333333333333), ('mmmu/history', 0.7333333333333333), ('mmmu/literature', 0.8), ('mmmu/manage', 0.5666666666666667), ('mmmu/marketing', 0.43333333333333335), ('mmmu/m

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
200,0.5556,0.5444,0.649,0.391,0.7937,0.4937,0.7255,0.68126,0.641,0.5379,0.7381
400,0.57,0.5656,0.6534,0.399,0.7895,0.4979,0.7258,0.68664,0.6334,0.5391,0.7353
600,0.5567,0.5433,0.6576,0.379,0.7947,0.4977,0.7262,0.68338,0.6451,0.5388,0.737
800,0.5756,0.5622,0.6862,0.435,0.8258,0.5386,0.7527,0.7053,0.6473,0.5585,0.7507
1000,0.5667,0.5511,0.6915,0.441,0.8229,0.5447,0.7517,0.70046,0.6548,0.5557,0.7557
1200,0.5833,0.5778,0.6882,0.443,0.8251,0.5338,0.753,0.69956,0.6509,0.5539,0.7495
1400,0.5633,0.5578,0.6576,0.39,0.8015,0.5053,0.7249,0.68468,0.637,0.5447,0.7389
1600,0.5811,0.5678,0.6634,0.387,0.8018,0.5018,0.7266,0.68846,0.6457,0.5496,0.7391
1800,0.5822,0.57,0.6844,0.479,0.8174,NaN,0.7503,0.70316,0.6526,0.5586,0.7516
2000,0.5522,0.5433,0.6628,0.378,0.8021,0.5066,0.7267,0.68474,0.642,0.5559,0.736


## Exp 28

In [74]:
eval_helper.run_eval_sweep(
    output_dir=f"/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28",
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR,
    aligner_parent_dir=ALIGNER_CODE_DIR,
)

New checkpoints: [6000]
Start eval for /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28/checkpoint-6000
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28/evals/eval_jobs_checkpoint-6000.json


In [79]:
eval_helper.get_eval_scores_all("/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28")

[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000, 4200, 4400, 4600, 4800, 5000, 5200, 5400, 5600, 5800, 6000]
Got result for mmmu - 200: [('mmmu/accounting', 0.5), ('mmmu/agriculture', 0.5666666666666667), ('mmmu/architecture_and_engineering', 0.43333333333333335), ('mmmu/art', 0.7), ('mmmu/art_theory', 0.8666666666666667), ('mmmu/basic_medical_science', 0.6666666666666666), ('mmmu/biology', 0.3333333333333333), ('mmmu/chemistry', 0.4), ('mmmu/clinical_medicine', 0.6333333333333333), ('mmmu/computer_science', 0.6), ('mmmu/design', 0.7), ('mmmu/diagnostics_and_laboratory_medicine', 0.4666666666666667), ('mmmu/economics', 0.5333333333333333), ('mmmu/electronics', 0.43333333333333335), ('mmmu/energy_and_power', 0.6), ('mmmu/finance', 0.4666666666666667), ('mmmu/geography', 0.5333333333333333), ('mmmu/history', 0.7333333333333333), ('mmmu/literature', 0.8333333333333334), ('mmmu/manage', 0.4666666666666667), ('mmmu/marketin

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
200,0.5689,0.5578,0.6852,0.411,0.8174,0.5189,0.7468,0.69688,0.6447,0.5587,0.743
400,0.5767,0.5544,0.6946,0.42,0.818,0.5345,0.7513,0.7067,0.6489,0.5459,0.7529
600,0.58,0.5689,0.6835,0.432,0.8235,0.5301,0.7491,0.70012,0.6515,0.5574,0.743
800,0.5756,0.5622,0.6862,0.435,0.8258,0.5386,0.7527,0.7053,0.6473,0.5585,0.7507
1000,0.5667,0.5511,0.6915,0.441,0.8229,0.5447,0.7517,0.70046,0.6548,0.5557,0.7557
1200,0.5833,0.5778,0.6882,0.443,0.8251,0.5338,0.753,0.69956,0.6509,0.5539,0.7495
1400,0.5967,0.5944,0.693,0.449,0.819,0.536,0.7514,0.7001,0.6499,0.5689,0.7511
1600,0.59,0.5833,0.6895,0.462,0.8167,0.5229,0.7529,0.69826,0.6523,0.5537,0.7524
1800,0.5822,0.57,0.6844,0.479,0.8174,NaN,0.7503,0.70316,0.6526,0.5586,0.7516
2000,0.5811,0.5656,0.69,0.465,0.819,0.5428,0.7523,0.70334,0.6562,0.5637,0.7527


## 504px exp 29

In [75]:
eval_helper.run_eval_sweep(
    output_dir=f"/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp28",
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR,
    aligner_parent_dir=ALIGNER_CODE_DIR,
)

New checkpoints: [4200, 4400, 4600]
Start eval for /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp28/checkpoint-4200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp28/evals/eval_jobs_checkpoint-4200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp28/checkpoint-4400
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp28/evals/eval_jobs_checkpoint-4400.json
Start eval for /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp28/checkpoint-4600
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp28/evals/eval_jobs_checkpoint-4600.json


In [48]:
# output_dir = "/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp29"
# c = 5200
# eval_helper.run_eval_plan(
#     eval_base_sbatch=EVAL_SBATCH,
#     aligner_parent_dir=ALIGNER_CODE_DIR,
#     eval_config_dir=EVAL_CONFIG_DIR,
#     checkpoint_dir=output_dir,
#     checkpoints=[c],
#     save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
#     benchmarks=["mmmu"],
#     rerun_if_exists=True,
#     print_cmd=True
# )

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3554613515.py, line 10)

In [78]:
eval_helper.get_eval_scores_all("/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp28")

[3400, 3600, 3800, 4000, 4200, 4400, 4600]
Got result for mmmu - 3400: [('mmmu/accounting', 0.4), ('mmmu/agriculture', 0.5666666666666667), ('mmmu/architecture_and_engineering', 0.3333333333333333), ('mmmu/art', 0.6666666666666666), ('mmmu/art_theory', 0.9), ('mmmu/basic_medical_science', 0.7), ('mmmu/biology', 0.43333333333333335), ('mmmu/chemistry', 0.36666666666666664), ('mmmu/clinical_medicine', 0.6666666666666666), ('mmmu/computer_science', 0.6333333333333333), ('mmmu/design', 0.8), ('mmmu/diagnostics_and_laboratory_medicine', 0.4), ('mmmu/economics', 0.8), ('mmmu/electronics', 0.36666666666666664), ('mmmu/energy_and_power', 0.4666666666666667), ('mmmu/finance', 0.5), ('mmmu/geography', 0.5), ('mmmu/history', 0.8333333333333334), ('mmmu/literature', 0.9), ('mmmu/manage', 0.43333333333333335), ('mmmu/marketing', 0.4666666666666667), ('mmmu/materials', 0.4), ('mmmu/math', 0.43333333333333335), ('mmmu/mechanical_engineering', 0.3333333333333333), ('mmmu/music', 0.26666666666666666), 

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
3400,0.5556,0.5489,0.6664,0.397,0.7983,0.5204,0.7225,0.68054,0.6387,0.553,0.7445
3600,0.5767,0.5567,0.6618,0.384,0.7986,0.512,0.7218,0.6834,0.6413,0.541,0.7362
3800,0.5611,0.5511,0.6628,0.382,0.7911,0.5221,0.722,0.68152,0.6434,0.5537,0.7413
4000,0.5678,0.5556,0.6646,0.385,0.7837,0.5197,0.721,0.68378,0.6377,0.5378,0.7423
4200,NaN,NaN,0.6566,0.385,0.8002,0.5192,0.7236,0.6838,0.6502,0.5479,0.7458
4400,NaN,NaN,0.6558,0.389,0.7957,0.5025,0.7237,0.6866,0.6371,0.5431,0.7443
4600,NaN,NaN,0.6639,0.386,0.8031,0.5193,0.721,0.68558,0.6365,0.5439,0.744


## 336px i18n

In [24]:
eval_helper.run_eval_sweep(
    output_dir=f"/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_64nodes_i18n",
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR,
    aligner_parent_dir=ALIGNER_CODE_DIR,
)

New checkpoints: []


In [19]:
eval_helper.get_eval_scores_all("/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_64nodes_i18n")

[5000, 5200, 5400, 5600, 5800, 6000, 6200, 6400]
Got result for mmmu - 5000: [('mmmu/accounting', 0.6), ('mmmu/agriculture', 0.5666666666666667), ('mmmu/architecture_and_engineering', 0.3), ('mmmu/art', 0.6666666666666666), ('mmmu/art_theory', 0.8666666666666667), ('mmmu/basic_medical_science', 0.6333333333333333), ('mmmu/biology', 0.3333333333333333), ('mmmu/chemistry', 0.43333333333333335), ('mmmu/clinical_medicine', 0.6333333333333333), ('mmmu/computer_science', 0.4666666666666667), ('mmmu/design', 0.6), ('mmmu/diagnostics_and_laboratory_medicine', 0.4), ('mmmu/economics', 0.7), ('mmmu/electronics', 0.4), ('mmmu/energy_and_power', 0.6333333333333333), ('mmmu/finance', 0.4), ('mmmu/geography', 0.6), ('mmmu/history', 0.7666666666666667), ('mmmu/literature', 0.8666666666666667), ('mmmu/manage', 0.5333333333333333), ('mmmu/marketing', 0.43333333333333335), ('mmmu/materials', 0.3), ('mmmu/math', 0.5), ('mmmu/mechanical_engineering', 0.4), ('mmmu/music', 0.3), ('mmmu/pharmacy', 0.7), ('mm

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
5000,0.5444,0.5433,0.6706,0.476,0.8488,0.7014,0.7256,0.66916,0.6587,0.5542,0.7887
5200,0.5422,0.5278,0.6631,0.524,0.8724,0.7788,0.728,0.66484,0.6527,0.5203,0.7947
5400,0.5444,0.5356,0.6418,0.521,0.8818,0.7923,0.7172,0.65898,0.6535,0.5298,0.7902
5600,0.5278,0.5211,0.6488,NaN,0.8951,0.7982,0.7123,0.65266,NaN,0.54,0.7879
5800,0.55,0.5389,0.6482,0.565,0.8909,0.801,0.7169,0.6532,0.6401,0.5225,0.7844
6000,0.5544,0.5522,0.6226,0.55,0.9028,0.8121,0.7098,0.65218,0.6485,0.5236,0.7864
6200,0.5478,0.5433,0.6321,0.564,0.8957,0.8031,0.7086,0.65124,0.644,0.5278,0.7886
6400,0.5489,0.5444,0.6457,0.581,0.9035,0.8084,0.7016,0.64498,0.6481,0.5346,0.7977


## Exp 30

In [4]:
eval_helper.run_eval_sweep(
    output_dir=f"/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30",
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR,
    aligner_parent_dir=ALIGNER_CODE_DIR,
    min_checkpoint=9200
)

New checkpoints: [6000, 6200, 6400, 6600, 6800, 7000, 7200, 7400, 7600, 7800, 8000, 8200, 8400, 8600, 8800, 9000, 9200, 9400, 9600, 9800, 10000, 10200]
Start eval for /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30/checkpoint-9200


FileNotFoundError: [Errno 2] No such file or directory: '/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30/evals/eval_jobs_checkpoint-9200.json'

In [8]:
EVAL_CONFIG_DIR

'/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval'

In [4]:
output_dir = "/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30"
c = 9200
eval_helper.run_eval_plan(
    eval_base_sbatch=EVAL_SBATCH,
    aligner_parent_dir=ALIGNER_CODE_DIR,
    eval_config_dir=EVAL_CONFIG_DIR,
    checkpoint_dir=output_dir,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
    # benchmarks=["mmmu"],
    rerun_if_exists=True,
    print_cmd=True
)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30 mmmu 9200
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30 docvqa 9200
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30 mathvista 9200
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_

In [9]:
eval_helper.get_eval_scores(output_dir, c)

{'mmmu': {'9200': 20362}, 'docvqa': {'9200': 20363}, 'mathvista': {'9200': 20364}, 'ai2d': {'9200': 20365}, 'chartqa': {'9200': 20366}, 'vqa': {'9200': 20367}, 'textvqa': {'9200': 20368}, 'infographics_w_ocr': {'9200': 20369}, 'infographics': {'9200': 20370}, 'mmbench': {'9200': 20371}}
Got result for mathvista - 9200: [('mathvista/accuracy', 0.532)]
Got result for chartqa - 9200: [('chartqa/accuracy', 0.5888241969056048)]
Got result for mmbench - 9200: [('mmbench/attribute_comparison', 0.7272727272727273), ('mmbench/attribute_recognition', 0.9295774647887324), ('mmbench/celebrity_recognition', 0.9595959595959596), ('mmbench/function_reasoning', 0.8481012658227848), ('mmbench/future_prediction', 0.6), ('mmbench/image_quality', 0.6341463414634146), ('mmbench/image_scene', 0.9519230769230769), ('mmbench/image_style', 0.9245283018867925), ('mmbench/image_topic', 0.8888888888888888), ('mmbench/nature_relation', 0.7916666666666666), ('mmbench/object_localization', 0.5432098765432098), ('mmb

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
9200,NaN,NaN,NaN,0.532,NaN,0.5888,NaN,NaN,NaN,NaN,0.7869


In [14]:
eval_helper.get_eval_jobs_record(
    "/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30", 
    9200
)

'/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30/evals/eval_jobs_checkpoint-9200.json'

In [10]:
output_dir = "/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30"
c = 9400
eval_helper.run_eval_plan(
    eval_base_sbatch=EVAL_SBATCH,
    aligner_parent_dir=ALIGNER_CODE_DIR,
    eval_config_dir="/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval",
    checkpoint_dir=output_dir,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
    # benchmarks=["mmmu"],
    rerun_if_exists=True,
    print_cmd=True
)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30 mmmu 9400
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30 docvqa 9400
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30 mathvista 9400
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_

# SFT

In [3]:
EVAL_SBATCH = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
ALIGNER_CODE_DIR="/fsx_0/user/tranx/eval_adel"
EVAL_CONFIG_DIR=f"/fsx_0/user/tranx/eval_adel/llm_mm_aligner/experiments/aws_adel/eval"

In [5]:
eval_helper.get_eval_jobs_record(output_dir, c)

'/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128/evals/eval_jobs_checkpoint-700.json'

In [18]:
ALIGNER_ADEL_PROD = '/fsx_0/user/tranx/adel_prod'

output_dir = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128"
c = 700
eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    aligner_parent_dir=ALIGNER_ADEL_PROD,
    eval_config_dir=f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft",
    checkpoint_dir=output_dir,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
    # benchmarks=["mmmu"],
    # benchmarks=["mathvista"],
    benchmarks=["ai2d"],
    rerun_if_exists=True,
    print_cmd=True
)

sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft/eval_ai2d.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128 ai2d 700
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128/evals/eval_jobs_checkpoint-700.json


# Eval Config

In [40]:
# Generate overwrite
train_config = utils.read_json("/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_mh19_stage2/stage2_MM9_70B_MH19_336px_128nodes_exp30.json")

for f in glob.glob("/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_mh19_stage2/eval_*.json"):
    eval_config = utils.read_json(f)
    
    benchmark_file = f.split("/")[-1]
    
    overwrite = eval_helper.get_eval_config_overwrite(train_config, eval_config)
    
    utils.save_json(overwrite, f"/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_mh19_stage2/eval_overwrite/overwrite_{benchmark_file}")

In [44]:
train_config.keys()

dict_keys(['scheduler_type', 'hpc_job_oncall', 'num_gpus', 'num_nodes', 'trainer_args', 'fsdp_config', 'datarecipe'])

In [10]:
# Generate eval config 
config_dir = "/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65"
train_config = utils.read_json(f"{config_dir}/sft_mm9_babyLora_16nodes.json")

# for b in eval_helper.ALL_BENCHMARKS:
for b in ["mmbench", "infographics_w_ocr", "docvqa"]:
    print(b)
    overwrite = utils.read_json(f"/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_mh19_stage2/eval_overwrite/overwrite_eval_{b}.json")
    
    eval_config = eval_helper.gen_eval_config(train_config, overwrite)
    
    utils.save_json(eval_config, f"/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_{b}.json")

mmbench
infographics_w_ocr
docvqa


## fb overwrite

In [31]:
data_files = {}
for f in glob.glob("/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_*.json"):
    conf = utils.read_json(f)
    bm = f.split("/")[-1].replace("eval_", "").replace(".json", "")
    # print(bm)
    data_files[bm] = {
        "train_file": conf["eval_args"]["train_file"],
        "validation_file": conf["eval_args"]["validation_file"]
    }

data_files

{'infographics_w_ocr': {'train_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl',
  'validation_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl'},
 'infographics': {'train_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl',
  'validation_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl'},
 'mmbench': {'train_file': '/fsx_0/dataset01/mmbench/processed_dev_20231212.json',
  'validation_file': '/fsx_0/dataset01/mmbench/processed_dev_20231212.json'},
 'mathvista': {'train_file': '/fsx_0/dataset01/mathvista/test.jsonl',
  'validation_file': '/fsx_0/dataset01/mathvista/test.jsonl'},
 'ai2d': {'train_file': '/fsx_0/dataset01/ai2d/ai2d_test.jsonl',
  'validation_file': '/fsx_0/dataset01/ai2d/ai2d_test.jsonl'},
 'docvqa': {'train_file': '/fsx_0/dataset01/docvqa/val.jsonl',
  'validation_file': '/fsx_0/dataset01/docvqa/val.jsonl'},
 'textvqa': {'train_file': '/fsx_0/dataset01/textvqa/text_vqa_val_set_updated.jsonl',
  'validation_file': '/fsx_0/dat

In [50]:
import copy 
from pprint import pprint

config_dir = "/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_fb"
train_config = utils.read_json(f"{config_dir}/sft_mm9_babyLora_16nodes_r26a.json")

for b in eval_helper.ALL_BENCHMARKS:
    print(b)
    overwrite_file = f"/fsx_0/user/tranx/experiments/eval/sft_eval_overwrite_fb/{b}.json"
    overwrite = utils.read_json(overwrite_file)
    
    eval_config = {
        "eval_only": True,
        "eval_args": copy.deepcopy(train_config["trainer_args"]),
        "fsdp_config": copy.deepcopy(train_config["fsdp_config"])
    }
    

    eval_config["eval_args"].update(overwrite["eval_args"])
    

    eval_config["eval_args"].update({
        "resume_from_checkpoint": "CHECKPOINT_PATH",
        "logging_dir": "LOGGING_DIR",
        "output_dir": "OUTPUT_DIR",
        "tb_logdir": "TB_LOGDIR",
        "eval_ckpt": "EVAL_CHECKPOINT",
        "train_file": data_files[b]["train_file"],
        "validation_file": data_files[b]["validation_file"]
    })
    
    # for k in ["per_device_eval_batch_size", 
    #           "batch_size_generation", 
    #           "max_seq_len",
    #           "max_length",
    #           "max_new_tokens"
    #           ]:
    #     if k in eval_config["eval_args"]:
    #         print(k, ":", eval_config["eval_args"][k])
    # print("--------------")
    # to fix OOM
    eval_config["eval_args"].update({
        "per_device_eval_batch_size": 1,
        "batch_size_generation": 1
    })
    
    utils.save_json(eval_config, f"{config_dir}/eval_{b}.json")
    

mmmu
docvqa
mathvista
ai2d
chartqa
vqa
textvqa
infographics_w_ocr
infographics
mmbench


In [51]:
# try a run
ALIGNER_ADEL_PROD_FSDP = '/fsx_0/user/tranx/adel_prod_fsdp'
ALIGNER_ADEL_PROD = '/fsx_0/user/tranx/adel_prod'
output_dir = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a"

c = 100
eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    eval_config_dir=f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65",
    checkpoint_dir=output_dir,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
    
    # # benchmarks=["mmmu"],
    # benchmarks=["mathvista"],
    # # benchmarks=["ai2d"],
    # benchmarks=[b for b in eval_helper.ALL_BENCHMARKS if b not in ["mmmu"]],
    
    rerun_if_exists=True,
    update_if_exists=True,
    print_cmd=True,
    print_job_dict=True
)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod_fsdp /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_fb/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a mmmu 100
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod_fsdp /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_fb/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a docvqa 100
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod_fsdp /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_fb/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a mathvista 100
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/e

In [52]:
eval_helper.get_eval_scores_all("/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


In [37]:
# try a run
ALIGNER_ADEL_PROD = '/fsx_0/user/tranx/adel_prod'
output_dir = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a"

c = 100
eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    aligner_parent_dir=ALIGNER_ADEL_PROD,
    eval_config_dir=f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_fb",
    checkpoint_dir=output_dir,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
    
    benchmarks=["mmmu"],
    # benchmarks=["mathvista"],
    # benchmarks=["ai2d"],
    
    rerun_if_exists=True,
    update_if_exists=True,
    print_cmd=True,
    print_job_dict=True
)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_fb/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a mmmu 100
OrderedDict([('mathvista', OrderedDict([('100', 21750)])), ('mmmu', {100: 21752})])
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/evals/eval_jobs_checkpoint-100.json


## try a run

In [69]:
# try a run
ALIGNER_ADEL_PROD = '/fsx_0/user/tranx/adel_prod'

output_dir = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes"

c = 3000
eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    aligner_parent_dir=ALIGNER_ADEL_PROD,
    eval_config_dir=f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65",
    checkpoint_dir=output_dir,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
    
    # benchmarks=["mmmu"],
    benchmarks=["mathvista"],
    # benchmarks=["ai2d"],
    
    rerun_if_exists=True,
    print_cmd=True,
    print_job_dict=True
)

sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes mathvista 3000
{'mathvista': {3000: 20495}}
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/evals/eval_jobs_checkpoint-3000.json


In [76]:
c = 900
eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    aligner_parent_dir=ALIGNER_ADEL_PROD,
    eval_config_dir=f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65",
    checkpoint_dir=output_dir,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
    
    benchmarks=["mmmu"],
    # benchmarks=["mathvista"],
    # benchmarks=["ai2d"],
    
    rerun_if_exists=True,
    print_cmd=True,
    print_job_dict=True
)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes mmmu 900
{'mmmu': {900: 20530}}
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/evals/eval_jobs_checkpoint-900.json


In [72]:
c = 1000
eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    aligner_parent_dir=ALIGNER_ADEL_PROD,
    eval_config_dir=f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65",
    checkpoint_dir=output_dir,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
    
    # benchmarks=["mmmu"],
    # benchmarks=["mathvista"],
    # benchmarks=["ai2d"],
    
    rerun_if_exists=True,
    print_cmd=True,
    print_job_dict=True
)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes mmmu 1000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes docvqa 1000
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes mathvista 1000
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_ev

In [74]:
eval_helper.get_eval_scores_all("/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes")

[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200]
{'mmmu': {'1000': 20512}, 'docvqa': {'1000': 20513}, 'mathvista': {'1000': 20514}, 'ai2d': {'1000': 20515}, 'chartqa': {'1000': 20516}, 'vqa': {'1000': 20517}, 'textvqa': {'1000': 20518}, 'infographics_w_ocr': {'1000': 20519}, 'infographics': {'1000': 20520}, 'mmbench': {'1000': 20521}}
Got result for mmmu - 1000: [('mmmu/accounting', 0.5333333333333333), ('mmmu/agriculture', 0.6666666666666666), ('mmmu/architecture_and_engineering', 0.3), ('mmmu/art', 0.7), ('mmmu/art_theory', 0.8333333333333334), ('mmmu/basic_medical_science', 0.6666666666666666), ('mmmu/biology', 0.3333333333333333), ('mmmu/chemistry', 0.36666666666666664), ('mmmu/clinical_medicine', 0.6333333333333333), ('mmmu/computer_science', 0.43333333333333335), ('mmmu/design', 0.7), ('mmmu/diagnostics_and_laboratory_medicine', 0.4333333333333333

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
1000,0.54,0.5256,NaN,NaN,NaN,0.561,NaN,NaN,NaN,NaN,NaN
1000,0.54,0.5256,NaN,NaN,NaN,0.561,NaN,NaN,NaN,NaN,NaN


In [78]:
c = 1000
eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    aligner_parent_dir=ALIGNER_ADEL_PROD,
    eval_config_dir=f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65",
    checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c",
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
    
    benchmarks=["mmmu"],
    # benchmarks=["mathvista"],
    # benchmarks=["ai2d"],
    
    rerun_if_exists=True,
    print_cmd=True,
    print_job_dict=True
)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c mmmu 1000
{'mmmu': {1000: 20550}}
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/evals/eval_jobs_checkpoint-1000.json


In [87]:
c = 1000
eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    aligner_parent_dir=ALIGNER_ADEL_PROD,
    eval_config_dir=f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65",
    checkpoint_dir=output_dir,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(output_dir, c),
    
    # benchmarks=["mmmu"],
    # benchmarks=["mathvista"],
    # benchmarks=["ai2d"],
    # benchmarks=["mmbench", "infographics_w_ocr", "docvqa"],
    # benchmarks=["mmbench", "infographics_w_ocr"],
    # benchmarks=["mmbench", "infographics_w_ocr"],
    benchmarks=["docvqa"],
    
    rerun_if_exists=True,
    print_cmd=True,
    print_job_dict=True
)

sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes docvqa 1000
{'docvqa': {1000: 20622}}
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/evals/eval_jobs_checkpoint-1000.json


# SFT Sweep

In [31]:
eval_helper.ALL_BENCHMARKS

['mmmu',
 'docvqa',
 'mathvista',
 'ai2d',
 'chartqa',
 'vqa',
 'textvqa',
 'infographics_w_ocr',
 'infographics',
 'mmbench']

In [33]:
ALIGNER_ADEL_PROD = '/fsx_0/user/tranx/adel_prod'
EVAL_SBATCH = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
EVAL_CONFIG_DIR_EMB65 = f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65"
EVAL_CONFIG_DIR_EMB65_FB = f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_fb"

EVAL_CONFIG_DIR_EMB129 = f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb129"


OUTPUT_SFT1 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes"
OUTPUT_SFT3 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e"
OUTPUT_SFT4 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c"

OUTPUT_SFT2 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128"
OUTPUT_SFT5 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_16nodes_latents128"

OUTPUT_SFT6 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a"
OUTPUT_SFT7 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26b"

# BENCHMARKS = ["mathvista", "vqa"]
BENCHMARKS = ["mmmu"]

## SFT1

In [37]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT1,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    aligner_parent_dir=ALIGNER_ADEL_PROD,
    # benchmarks=["mmmu", "chartqa", "mmbench"],
    benchmarks=BENCHMARKS,
    update_if_exists=True
)

New checkpoints: [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/evals/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/evals/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/evals/eval_jobs_checkpoint-300.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/checkpoint-400
eval jobs saved to: /

In [40]:
eval_helper.get_eval_scores_all(OUTPUT_SFT1)

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,0.55,0.5322,NaN,0.501,NaN,0.6372,NaN,NaN,NaN,NaN,0.7722
200,0.5456,0.5278,NaN,0.492,NaN,0.6097,NaN,NaN,NaN,NaN,0.7733
300,0.5422,0.5267,NaN,NaN,NaN,0.5957,NaN,NaN,NaN,NaN,0.7652
400,0.5433,0.5267,NaN,0.495,NaN,0.5745,NaN,NaN,NaN,NaN,0.7624
500,0.5478,0.5311,NaN,0.503,NaN,0.5808,NaN,NaN,NaN,NaN,0.7699
600,0.5478,0.53,NaN,NaN,NaN,0.5804,NaN,NaN,NaN,NaN,0.7656
700,0.5456,0.53,NaN,NaN,NaN,0.5823,NaN,NaN,NaN,NaN,0.7724
800,0.55,0.5333,NaN,0.515,NaN,0.5719,NaN,NaN,NaN,NaN,0.771
900,0.5411,0.5256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,0.507,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df1 = eval_helper.get_eval_scores_all(OUTPUT_SFT1)
df1 = df1.sort_values(by=['mmmu_v2'], ascending=False)
df1.head()

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
2200,0.5556,0.5389,NaN,NaN,NaN,0.5485,NaN,NaN,NaN,NaN,0.7636
100,0.55,0.5322,NaN,0.503,NaN,0.6372,NaN,NaN,NaN,NaN,0.7722
800,0.55,0.5333,NaN,NaN,NaN,0.5719,NaN,NaN,NaN,NaN,0.771
500,0.5478,0.5311,NaN,NaN,NaN,0.5808,NaN,NaN,NaN,NaN,0.7699
600,0.5478,0.53,NaN,NaN,NaN,0.5804,NaN,NaN,NaN,NaN,0.7656


## SFT3

In [38]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT3,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    aligner_parent_dir=ALIGNER_ADEL_PROD,
    # benchmarks=["mmmu", "chartqa", "mmbench"],
    benchmarks=BENCHMARKS,
    update_if_exists=True
)

New checkpoints: [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e/evals/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e/evals/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e/evals/eval_jobs_checkpoint-300.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e/checkpoint-400
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-7

In [41]:
eval_helper.get_eval_scores_all(OUTPUT_SFT3)

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,0.5522,0.5322,NaN,NaN,NaN,0.6306,NaN,NaN,NaN,NaN,0.7514
200,0.5522,0.5333,NaN,NaN,NaN,0.6071,NaN,NaN,NaN,NaN,0.7547
300,0.5467,0.5267,NaN,0.524,NaN,0.5938,NaN,NaN,NaN,NaN,0.7629
400,0.5444,0.5244,NaN,NaN,NaN,0.5776,NaN,NaN,NaN,NaN,0.7648
500,0.5544,0.5344,NaN,NaN,NaN,0.5695,NaN,NaN,NaN,NaN,0.764
600,0.5422,0.5233,NaN,0.535,NaN,0.5775,NaN,NaN,NaN,NaN,0.7678
700,0.5467,0.5278,NaN,NaN,NaN,0.5772,NaN,NaN,NaN,NaN,0.764
800,0.5411,0.5267,NaN,NaN,NaN,0.5749,NaN,NaN,NaN,NaN,0.765
900,0.5522,0.5333,NaN,0.532,NaN,0.5712,NaN,NaN,NaN,NaN,0.7668
1000,0.5467,0.53,NaN,NaN,NaN,0.5735,NaN,NaN,NaN,NaN,0.7667


In [29]:
df3 = eval_helper.get_eval_scores_all(OUTPUT_SFT3)
df3 = df3.sort_values(by=['mmmu_v2'], ascending=False)
df3.head()

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
1100,0.5556,0.5422,NaN,NaN,NaN,0.5715,NaN,NaN,NaN,NaN,0.7631
500,0.5544,0.5344,NaN,NaN,NaN,0.5695,NaN,NaN,NaN,NaN,0.764
200,0.5522,0.5333,NaN,NaN,NaN,0.6071,NaN,NaN,NaN,NaN,0.7547
900,0.5522,0.5333,NaN,NaN,NaN,0.5712,NaN,NaN,NaN,NaN,0.7668
100,0.5522,0.5322,NaN,NaN,NaN,0.6306,NaN,NaN,NaN,NaN,0.7514


## SFT4

In [39]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT4,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    aligner_parent_dir=ALIGNER_ADEL_PROD,
    # benchmarks=["mmmu", "chartqa", "mmbench"],
    benchmarks=BENCHMARKS,
    update_if_exists=True
)

New checkpoints: [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c/evals/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c/evals/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c/evals/eval_jobs_checkpoint-300.json
Start eval for /fsx_0/checkpoints/tranx/MM9

In [42]:
eval_helper.get_eval_scores_all(OUTPUT_SFT4)

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,0.54,0.5233,NaN,NaN,NaN,0.6274,NaN,NaN,NaN,NaN,0.7579
200,0.5311,0.5122,NaN,NaN,NaN,0.5996,NaN,NaN,NaN,NaN,0.7507
300,0.5344,0.5167,NaN,NaN,NaN,0.5888,NaN,NaN,NaN,NaN,0.7463
400,0.5367,0.5167,NaN,NaN,NaN,0.5769,NaN,NaN,NaN,NaN,0.7504
500,0.5378,0.5233,NaN,0.537,NaN,0.5724,NaN,NaN,NaN,NaN,0.7599
600,0.5411,0.5233,NaN,NaN,NaN,0.579,NaN,NaN,NaN,NaN,0.7636
700,0.5367,0.5256,NaN,NaN,NaN,0.5777,NaN,NaN,NaN,NaN,0.7653
800,0.5433,0.5333,NaN,0.538,NaN,0.5859,NaN,NaN,NaN,NaN,0.7627
900,0.5533,0.5422,NaN,0.536,NaN,0.5707,NaN,NaN,NaN,NaN,0.7648
1000,0.5378,0.5311,NaN,NaN,NaN,0.5742,NaN,NaN,NaN,NaN,0.7615


In [30]:
df4 = eval_helper.get_eval_scores_all(OUTPUT_SFT4)
df4 = df4.sort_values(by=['mmmu_v2'], ascending=False)
df4.head()

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
900,0.5533,0.5422,NaN,NaN,NaN,0.5707,NaN,NaN,NaN,NaN,0.7648
4100,0.55,0.5311,NaN,NaN,NaN,0.5863,NaN,NaN,NaN,NaN,0.7651
1100,0.55,0.5378,NaN,NaN,NaN,0.5763,NaN,NaN,NaN,NaN,0.756
4300,0.5489,0.5378,NaN,NaN,NaN,0.5831,NaN,NaN,NaN,NaN,0.7615
4400,0.5444,0.5378,NaN,NaN,NaN,0.5802,NaN,NaN,NaN,NaN,0.7615


## SFT2

In [99]:
# eval_helper.run_eval_sweep(
#     output_dir=OUTPUT_SFT2,
#     eval_sbatch=EVAL_SBATCH,
#     eval_config_dir=EVAL_CONFIG_DIR_EMB129,
#     aligner_parent_dir=ALIGNER_ADEL_PROD,
#     benchmarks=["mmmu", "chartqa", "mmbench"],
#     rerun_if_exists=True
# )

New checkpoints: [100, 200, 300, 400, 500, 600, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128/evals/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128/evals/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes_latents128/evals/eval_jobs_checkpoint-300.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336

In [18]:
eval_helper.get_eval_scores_all(OUTPUT_SFT2)

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7028
200,NaN,NaN,NaN,NaN,NaN,0.5924,NaN,NaN,NaN,NaN,0.718
300,NaN,NaN,NaN,NaN,NaN,0.5785,NaN,NaN,NaN,NaN,0.73
400,NaN,NaN,NaN,NaN,NaN,0.5866,NaN,NaN,NaN,NaN,0.741
500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7516
600,NaN,NaN,NaN,NaN,NaN,0.575,NaN,NaN,NaN,NaN,0.7553
800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7528
900,NaN,NaN,NaN,NaN,NaN,0.5825,NaN,NaN,NaN,NaN,0.7557
1000,NaN,NaN,NaN,NaN,NaN,0.5705,NaN,NaN,NaN,NaN,0.7538
1100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7661


## SFT5

In [100]:
# eval_helper.run_eval_sweep(
#     output_dir=OUTPUT_SFT5,
#     eval_sbatch=EVAL_SBATCH,
#     eval_config_dir=EVAL_CONFIG_DIR_EMB129,
#     aligner_parent_dir=ALIGNER_ADEL_PROD,
#     benchmarks=["mmmu", "chartqa", "mmbench"],
#     rerun_if_exists=True
# )

New checkpoints: [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_16nodes_latents128/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_16nodes_latents128/evals/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_16nodes_latents128/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_16nodes_latents128/evals/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_16nodes_latents128/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_16nodes_latents128/evals/eval_jobs_checkpoint-300.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800

In [23]:
eval_helper.get_eval_scores_all(OUTPUT_SFT5)

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,NaN,NaN,NaN,NaN,NaN,0.5885,NaN,NaN,NaN,NaN,0.7031
200,NaN,NaN,NaN,NaN,NaN,0.5889,NaN,NaN,NaN,NaN,0.729
300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.7386
400,NaN,NaN,NaN,NaN,NaN,0.5874,NaN,NaN,NaN,NaN,0.7352
500,NaN,NaN,NaN,NaN,NaN,0.5835,NaN,NaN,NaN,NaN,0.7371
600,NaN,NaN,NaN,NaN,NaN,0.5844,NaN,NaN,NaN,NaN,0.7552
700,NaN,NaN,NaN,NaN,NaN,0.5968,NaN,NaN,NaN,NaN,0.7486
800,NaN,NaN,NaN,NaN,NaN,0.5924,NaN,NaN,NaN,NaN,0.7522
900,NaN,NaN,NaN,NaN,NaN,0.5844,NaN,NaN,NaN,NaN,0.7587
1000,NaN,NaN,NaN,NaN,NaN,0.5786,NaN,NaN,NaN,NaN,0.7501


# EVAL FSDP

In [4]:
ALIGNER_ADEL_PROD_FSDP = '/fsx_0/user/tranx/adel_prod_fsdp'
EVAL_SBATCH = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
EVAL_CONFIG_DIR_EMB65 = f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65"
EVAL_CONFIG_DIR_EMB129 = f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb129"

In [9]:
c = 100
eval_helper.run_eval_plan(
    eval_base_sbatch=EVAL_SBATCH,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    checkpoint_dir=OUTPUT_SFT1,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(OUTPUT_SFT1, c),
    
    # benchmarks=["mmmu"],
    # benchmarks=["mathvista"],
    # benchmarks=["ai2d"],
    # benchmarks=["mmbench", "infographics_w_ocr", "docvqa"],
    benchmarks=["mmbench", "infographics_w_ocr"],
    # benchmarks=["mmbench", "infographics_w_ocr"],
    # benchmarks=["docvqa"],
    
    update_if_exists=True,
    print_cmd=True,
    print_job_dict=True
)

sbatch --parsable --job-name=eval_mmbench /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod_fsdp /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_mmbench.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes mmbench 100
sbatch --parsable --job-name=eval_infographics_w_ocr /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod_fsdp /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_infographics_w_ocr.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes infographics_w_ocr 100
OrderedDict([('mmmu', OrderedDict([('100', 20625)])), ('chartqa', OrderedDict([('100', 20626)])), ('mmbench', {100: 21185}), ('docvqa', OrderedDict([('100', 21184)])), ('infographics_w_ocr', {100: 21186})])
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/evals/eval_jobs_checkpoint-100.json


In [16]:
# redo_benchmarks = [
#  'docvqa',
#  'mathvista',
#  'ai2d',
#  'vqa',
#  'textvqa',
#  'infographics_w_ocr',
#  'infographics',
#  'mmbench'
#  ]

redo_benchmarks = [
 'docvqa',
 'ai2d',
#  'vqa',
 'textvqa',
 'infographics_w_ocr',
 'infographics',
#  'mmbench'
 ]

c = 100
eval_helper.run_eval_plan(
    eval_base_sbatch=EVAL_SBATCH,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    checkpoint_dir=OUTPUT_SFT1,
    checkpoints=[c],
    save_eval_jobs=eval_helper.get_eval_jobs_record(OUTPUT_SFT1, c),
    benchmarks=redo_benchmarks,
    update_if_exists=True,
    print_cmd=True,
    print_job_dict=True
)

sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod_fsdp /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes docvqa 100
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod_fsdp /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_ai2d.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes ai2d 100
sbatch --parsable --job-name=eval_textvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh /fsx_0/user/tranx/adel_prod_fsdp /fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_textvqa.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes textvqa 100
sbatch --parsable --job-name=eval_infographics_w_ocr /fsx_0/user/tranx/exper